In [1]:
import pandas as pd
from datetime import datetime

# --- 1. Load Both Datasets ---
try:
    customers_df = pd.read_csv('AWCustomers.csv')
    sales_df = pd.read_csv('AWSales.csv')
    print("✅ Successfully loaded AWCustomers.csv and AWSales.csv")
except FileNotFoundError as e:
    print(f"❌ Error: Could not find a file. Make sure both CSVs are in the directory. Details: {e}")
    exit()

# --- 2. Create the 'BikeBuyer' column ---
# Get a unique list of customer IDs from the sales data
customers_who_bought = sales_df['CustomerID'].unique()

# Create the 'BikeBuyer' column in the customers dataframe.
# If a customer's ID is in our list of buyers, set BikeBuyer to 1, otherwise 0.
customers_df['BikeBuyer'] = customers_df['CustomerID'].apply(
    lambda id: 1 if id in customers_who_bought else 0
)
print("✅ 'BikeBuyer' column created successfully.")

# --- 3. Create the 'Age' column from 'BirthDate' ---
# Convert the 'BirthDate' column to a proper date format
customers_df['BirthDate'] = pd.to_datetime(customers_df['BirthDate'])

# Calculate age based on today's date
today = datetime.now()
customers_df['Age'] = customers_df['BirthDate'].apply(
    lambda birth_date: today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
)
print("✅ 'Age' column calculated successfully.")


# --- 4. Select the Final, Correct Features ---
# This is our corrected list of columns that actually exist in the data
final_selected_features = [
    'MaritalStatus', 'Gender', 'YearlyIncome', 'TotalChildren',
    'NumberChildrenAtHome', 'Education', 'Occupation',
    'HomeOwnerFlag', 'NumberCarsOwned', 'Age', # Note: 'Region' and 'CommuteDistance' were removed
    'BikeBuyer' # Our newly created target variable
]

# Create our final, clean DataFrame
df = customers_df[final_selected_features]

# --- 5. Display and Save the Clean Data ---
print("\n--- Success! Final Clean Data Frame is Ready ---")
print(df.head())

print("\nInformation about the final DataFrame:")
df.info()

# Save the cleaned dataframe to a new CSV file for the next steps
df.to_csv('cleaned_customer_data.csv', index=False)
print("\n✅ Final cleaned data has been saved to 'cleaned_customer_data.csv'")

✅ Successfully loaded AWCustomers.csv and AWSales.csv
✅ 'BikeBuyer' column created successfully.
✅ 'Age' column calculated successfully.

--- Success! Final Clean Data Frame is Ready ---
  MaritalStatus Gender  YearlyIncome  TotalChildren  NumberChildrenAtHome  \
0             M      M         81916              1                     0   
1             M      M         81076              2                     1   
2             S      F         86387              0                     0   
3             M      M         61481              2                     1   
4             S      M         51804              0                     0   

         Education      Occupation  HomeOwnerFlag  NumberCarsOwned  Age  \
0        Bachelors        Clerical              1                3   37   
1  Partial College        Clerical              1                2   53   
2        Bachelors        Clerical              0                3   39   
3  Partial College  Skilled Manual              1 

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the cleaned data from Part I
try:
    df = pd.read_csv('cleaned_customer_data.csv')
    print("✅ Cleaned data loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'cleaned_customer_data.csv' not found. Please make sure you ran the code from Part I.")
    exit()

# --- (a) Handling Null Values ---
# First, let's check if we have any missing values
print("\nMissing values before handling:")
print(df.isnull().sum())

# We will fill any missing values using a simple strategy:
# - For numeric columns, use the median (the middle value).
# - For text (categorical) columns, use the mode (the most common value).
for column in df.columns:
    if df[column].dtype == 'object':
        # It's a categorical column
        df[column].fillna(df[column].mode()[0], inplace=True)
    else:
        # It's a numeric column
        df[column].fillna(df[column].median(), inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())
print("✅ Null values handled.")


# --- (c) Discretization (Binning) on Continuous Attributes ---
# We'll group 'Age' and 'YearlyIncome' into a few categories to make patterns easier to find.
# Binning 'Age'
df['AgeBin'] = pd.cut(df['Age'], 
                      bins=[0, 30, 55, 100], 
                      labels=['Young', 'Middle-Aged', 'Senior'])

# Binning 'YearlyIncome'
df['IncomeBin'] = pd.cut(df['YearlyIncome'], 
                         bins=4, 
                         labels=['Low', 'Medium', 'High', 'Very High'])

print("\n✅ 'Age' and 'YearlyIncome' have been binned.")
# We can drop the original 'Age' and 'YearlyIncome' columns now, but we'll keep them for the normalization step next.


# --- (e) Binarization (One-Hot Encoding) ---
# We convert categorical columns (like 'Gender': M/F) into numeric ones (like 'Gender_M': 0/1).
# This includes our newly binned columns.
df_processed = pd.get_dummies(df, columns=[
    'MaritalStatus', 'Gender', 'Education', 'Occupation', 'AgeBin', 'IncomeBin'
], drop_first=True) # drop_first=True is a good practice to avoid redundant columns.

print("✅ Categorical features converted to numeric using One-Hot Encoding.")


# --- (b) & (d) Normalization ---
# Now, we scale all numeric features to be between 0 and 1.
# This ensures that features with large values (like YearlyIncome) don't dominate
# features with small values (like TotalChildren).
# We should not scale our target variable 'BikeBuyer' or binary flags.
scaler = MinMaxScaler()

# Select only the original numeric columns that need scaling
columns_to_scale = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'NumberCarsOwned', 'Age']
df_processed[columns_to_scale] = scaler.fit_transform(df_processed[columns_to_scale])

print("✅ Numeric features normalized.")


# --- Final Transformed Data ---
print("\n--- Success! Data is now fully preprocessed. ---")
print("First 5 rows of the final DataFrame:")
print(df_processed.head())

# Save the fully processed data for the final part of the assignment
df_processed.to_csv('processed_customer_data.csv', index=False)
print("\n✅ Fully processed data has been saved to 'processed_customer_data.csv'")

✅ Cleaned data loaded successfully.

Missing values before handling:
MaritalStatus           0
Gender                  0
YearlyIncome            0
TotalChildren           0
NumberChildrenAtHome    0
Education               0
Occupation              0
HomeOwnerFlag           0
NumberCarsOwned         0
Age                     0
BikeBuyer               0
dtype: int64

Missing values after handling:
MaritalStatus           0
Gender                  0
YearlyIncome            0
TotalChildren           0
NumberChildrenAtHome    0
Education               0
Occupation              0
HomeOwnerFlag           0
NumberCarsOwned         0
Age                     0
BikeBuyer               0
dtype: int64
✅ Null values handled.

✅ 'Age' and 'YearlyIncome' have been binned.
✅ Categorical features converted to numeric using One-Hot Encoding.
✅ Numeric features normalized.

--- Success! Data is now fully preprocessed. ---
First 5 rows of the final DataFrame:
   YearlyIncome  TotalChildren  NumberChildren

/var/folders/vp/mhbkf2gs397d1pq5hhxflp6h0000gn/T/ipykernel_24144/2176540745.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
/var/folders/vp/mhbkf2gs397d1pq5hhxflp6h0000gn/T/ipykernel_24144/2176540745.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the cleaned data from Part I (before normalization)
try:
    df_cleaned = pd.read_csv('cleaned_customer_data.csv')
    print("\n✅ Cleaned data (for correlation) loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'cleaned_customer_data.csv' not found.")
    exit()

# Select the two columns we want to analyze
correlation_df = df_cleaned[['Age', 'YearlyIncome']]

# Calculate the correlation matrix
correlation_matrix = correlation_df.corr()

# Get the specific correlation value between Age and YearlyIncome
correlation_value = correlation_matrix.loc['Age', 'YearlyIncome']

print("\n--- Correlation Analysis ---")
print(f"The Pearson Correlation between Age and Yearly Income is: {correlation_value:.4f}")

# --- Visualize the Correlation with a Heatmap ---
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap: Age vs. Yearly Income')
plt.savefig('correlation_heatmap.png') # Save the plot to a file
plt.close()

print("\n✅ Correlation heatmap has been saved as 'correlation_heatmap.png'")


✅ Cleaned data (for correlation) loaded successfully.

--- Correlation Analysis ---
The Pearson Correlation between Age and Yearly Income is: 0.0263

✅ Correlation heatmap has been saved as 'correlation_heatmap.png'
